In [ ]:
# notebooks/4_optimization.ipynb

import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit

# 1. Veriyi Oku
# Feature Engineering adımında oluşturduğumuz dosyayı yüklüyoruz
df = pd.read_csv('../data/atm_processed.csv')

# 2. Train - Test Ayrımı (Zaman Serisi Mantığıyla)
split_point = int(len(df) * 0.8)
train_df = df.iloc[:split_point]
test_df = df.iloc[split_point:]

# 3. Features ve Target Tanımlama
features = ['hour', 'day_of_week', 'day_of_month', 'is_weekend', 
            'lag_1', 'lag_24', 'rolling_mean_3', 'rolling_mean_24']
target = 'totalOutcome'

X_train = train_df[features]
y_train = train_df[target]
X_test = test_df[features]
y_test = test_df[target]

print(f"Eğitim Verisi Hazır: {X_train.shape}")

# 4. Optimizasyon Ayarları
# Modelin deneyeceği parametre havuzu
param_grid = {
    'n_estimators': [100, 200, 500, 1000],   # Ağaç sayısı
    'max_depth': [3, 5, 7, 10],              # Ağaç derinliği (karmaşıklık)
    'learning_rate': [0.01, 0.05, 0.1, 0.2], # Öğrenme hızı
    'subsample': [0.7, 0.8, 1.0],            # Her ağaç için kullanılacak veri oranı
    'colsample_bytree': [0.7, 0.8, 1.0]      # Her ağaç için kullanılacak özellik oranı
}

# Modeli Oluştur
model = xgb.XGBRegressor(random_state=42)

# TimeSeriesSplit kullanarak Cross Validation yapıyoruz (Zaman serisi olduğu için önemli!)
# Bu sayede model "geleceği görmeden" geçmiş verilerle test edilir.
tscv = TimeSeriesSplit(n_splits=3)

print("Optimizasyon başlıyor... Bu işlem 1-2 dakika sürebilir.")

# RandomizedSearchCV: Rastgele parametreler seçip dener (GridSearch'ten daha hızlıdır)
search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_grid,
    n_iter=15, # 15 farklı kombinasyon deneyecek
    cv=tscv,   # Zaman serisi validasyonu
    scoring='neg_root_mean_squared_error', # Hatayı minimize etmeye çalışır
    verbose=1,
    random_state=42,
    n_jobs=-1  # Tüm işlemci çekirdeklerini kullan
)

search.fit(X_train, y_train)

# 5. Sonuçları Raporla
print("\n------------------------------------------")
print(f"EN İYİ PARAMETRELER: {search.best_params_}")
print(f"EN İYİ SKOR (RMSE): {-search.best_score_:.2f}")
print("------------------------------------------")

# 6. En İyi Modeli Kaydet
# Bu modeli daha sonra app.py'da kullanmak üzere kaydediyoruz
best_model = search.best_estimator_

# Modeli tüm eğitim verisiyle tekrar eğitmek iyi bir pratiktir (isteğe bağlı)
best_model.fit(X_train, y_train)

best_model.save_model('../models/atm_optimized_model.json')
print("✅ Optimize edilmiş model kaydedildi: ../models/atm_optimized_model.json")

NameError: name 'X_train' is not defined